# Temporarily Generate Data for Verifier

In [2]:
import os; os.chdir("../")
import sys; sys.path.append("scripts")

In [3]:
import re
from utils import _extract_answer
from utils import HF_NAME_MAP
from transformers import AutoTokenizer
from datasets import load_from_disk, concatenate_datasets, Dataset
from collections import defaultdict

In [4]:
model_name = "sft_llama-8b"
task_name = "gsm8k"
model_path = f"models/{model_name}_{task_name}"

model_type, pt_name = model_name.split("_")
hf_name = HF_NAME_MAP[pt_name]
tokenizer = AutoTokenizer.from_pretrained(hf_name)

In [5]:
files = [f for f in os.listdir(f"data") if model_name in f]

In [6]:
dset = load_from_disk("data/sft_llama-1b_gsm8k_false_200_0")
# dset = load_from_disk("data/sft_llama-8b_gsm8k_false_100_1")
new_q = [q[0] for q in dset["question"]]
new_dset = {
    "question": new_q,
    "answer": dset['answer']
}

new_dset = Dataset.from_dict(new_dset)
# new_dset.save_to_disk("data/sft_llama-8b_gsm8k_correct_100_1")
# new_dset.save_to_disk("data/sft_llama-8b_gsm8k_false_100_1")

In [10]:
inst = "Please calculate the solution step-by-step and conclude the answer with \n#### followed by the result.\nQuestion: "
dset = load_from_disk("data/sft_llama-1b_gsm8k_false_200_0")
q_str = tokenizer.batch_decode(dset['question'], skip_special_tokens=True)
q_str = [q.split(inst)[-1] for q in q_str]
new_d = {
    "question": q_str,
    "answer": dset['answer']
}
new_dset = Dataset.from_dict(new_d)
print(len(new_dset))
print(new_dset[0])
new_dset.save_to_disk("data/sft_llama-1b_gsm8k_false_200_0")

# inst = "Please calculate the solution step-by-step and conclude the answer with \n#### followed by the result.\nQuestion: "
# dset = load_from_disk("data/sft_llama-8b_gsm8k_false_1900_0")
# q_str = tokenizer.batch_decode(dset['question'], skip_special_tokens=True)
# q_str = [q.split(inst)[-1] for q in q_str]
# new_d = {
#     "question": q_str,
#     "answer": dset['answer']
# }
# new_dset = Dataset.from_dict(new_d)
# print(len(new_dset))
# print(new_dset[0])

3047
{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'The number of clips sold to friends during April and May totaled 48 * 2 = <<48*2=96>>96.\nSo, after selling half of the clips in May, Natalia had 96 / 2 = <<96/2=48>>48 clips sold.\n#### 48'}


Saving the dataset (0/1 shards):   0%|          | 0/3047 [00:00<?, ? examples/s]

In [40]:
correct_set = load_from_disk("data/sft_llama-8b_gsm8k_correct_100_1")
false_set = load_from_disk("data/sft_llama-8b_gsm8k_false_100_1")
print(len(correct_set), len(false_set))

unique_question = list(set(correct_set['question']) | set(false_set['question']))

249 1367


In [69]:
import random

# Convert to dictionaries with sets
correct_answers = defaultdict(set)
for key, value in zip(correct_set["question"], correct_set["answer"]):
    correct_answers[key].add(value)
correct_answers = dict(correct_answers)

false_answers = defaultdict(set)
for key, value in zip(false_set["question"], false_set["answer"]):
    false_answers[key].add(value)
false_answers = dict(false_answers)

print(f"Correct questions: {len(correct_answers)}, False questions: {len(false_answers)}")

# Generate result
result = set()  # Use a set to avoid duplicates
for question, _correct_set in correct_answers.items():
    if question in false_answers:
        _false_set = false_answers[question]
        
        # Generate multiple unique pairs for each question
        for _ in range(50):  # Limit number of pairs per question
            chosen = random.choice(list(_correct_set))  # Always pick from correct answers
            rejected = random.choice(list(_false_set))  # Always pick from false answers
            pair = (question, chosen, rejected)  # Tuple ensures uniqueness in set
            result.add(pair)

# Convert set back to list of dictionaries
final_result = {
    "prompt": [],
    "chosen": [],
    "rejected": []
}

for entry in result:
    final_result["prompt"].append(entry[0])
    final_result["chosen"].append(entry[1])
    final_result["rejected"].append(entry[2])

print("Data size after filtering:", len(final_result['prompt']))
# # Output the results
# for entry in final_result:
#     print(f"Prompt: {entry['prompt']}")
#     print(f"Chosen: {entry['chosen']}")
#     print(f"Rejected: {entry['rejected']}")
#     print()

Correct questions: 73, False questions: 101
Data size after filtering: 1883


In [70]:
Dataset.from_dict(final_result).train_test_split(test_size=0.1).save_to_disk(f"data/verifier_dataset2")

Saving the dataset (0/1 shards):   0%|          | 0/1694 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/189 [00:00<?, ? examples/s]

In [10]:
# inst = "Please calculate the solution step-by-step and conclude the answer with \n#### followed by the result.\nQuestion: "
# correct_sets = []
# _correct_sets = [load_from_disk(f"data/{f}") for f in files if "correct" in f]
# dset_names = [f for f in files if "correct" in f]
# for d in _correct_sets:
#     if isinstance(d['question'][0][0], str):
#         correct_sets.append(d)
#     elif isinstance(d['question'][0][0], int):
#         q_str = tokenizer.batch_decode(d['question'], skip_special_tokens=True)
#         q_str = [q.split(inst)[-1] for q in q_str]
#         new_d = {
#             "question": q_str,
#             "answer": d['answer']
#         }
#         correct_sets.append(Dataset.from_dict(new_d))
#     else:
#         raise ValueError("Unknown question type")
# for idx in range(len(correct_sets)):
#     print(dset_names[idx])
#     print("\n")
#     print(correct_sets[idx]['question'][0])
#     print("\n")
#     print(correct_sets[idx]['answer'][0])
#     print("\n")

In [13]:
correct_set = concatenate_datasets(correct_sets)

ValueError: The features can't be aligned because the key question of features {'question': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'answer': Value(dtype='string', id=None)} has unexpected type - Sequence(feature=Value(dtype='string', id=None), length=-1, id=None) (expected either Value(dtype='string', id=None) or Value("null").

In [51]:
def evaluate(log_file_path, data="latest"):
    if data not in ["latest", "all"]:
        raise ValueError("data should be either 'latest' or 'all'")
    # 로그 데이터를 저장할 리스트
    parsed_data = []

    # 정규식을 사용하여 Question, Prediction, Answer를 파싱
    pattern_iteration_start = re.compile(r'Starting Evaluate script')
    pattern_question = re.compile(r'Question:\s*\[(.+?)\](?=\s*\[|$)', re.DOTALL)
    pattern_prediction = re.compile(r'Prediction:\s*\[(.+?)\](?=\s*\[|$)', re.DOTALL)
    pattern_answer = re.compile(r'Answer:\s*\[(.+?)\](?=\s*\[|$)', re.DOTALL)

    # 로그 파일 읽기 (예: 'log.txt'로 저장된 경우)
    with open(log_file_path, "r") as log_file:
        current_entry = {"question": None, "prediction": None, "answer": None}
        # 최근 iteration 데이터를 저장할 변수
        if data == "latest":
            current_iteration_data = []
            for line in log_file:
                # Iteration 시작 감지
                if pattern_iteration_start.search(line):
                    # 기존 iteration 데이터를 가장 최근 데이터로 갱신
                    current_iteration_data = []
                current_iteration_data.append(line)
        else:
            current_iteration_data = log_file.readlines()

        for line in current_iteration_data:
            question_match = pattern_question.search(line)
            if question_match:
                current_entry["question"] = question_match.group(1).strip("[]").strip('"')
                continue

            # Prediction 파싱
            prediction_match = pattern_prediction.search(line)
            if prediction_match:
                current_entry["prediction"] = prediction_match.group(1).strip("[]")
                continue

            # Answer 파싱
            answer_match = pattern_answer.search(line)
            if answer_match:
                current_entry["answer"] = answer_match.group(1).strip("[]").strip("'")
                # 완전한 데이터가 준비되면 저장
                if current_entry["question"] and current_entry["prediction"] and current_entry["answer"]:
                    parsed_data.append(current_entry)
                    current_entry = {"question": None, "prediction": None, "answer": None}

    # 결과 출력
    res = []
    answer_pat = re.compile(r"Answer:\s", re.IGNORECASE)

    for i, entry in enumerate(parsed_data):
        try:
            pn, gn = _extract_answer(entry["prediction"], entry["answer"])
            pred_loc = answer_pat.search(entry["prediction"])
            res.append({
                "question": entry["question"],
                "answer" : entry["answer"],
                "pred" : entry["prediction"][pred_loc.end():],
                # "pred" : entry["prediction"],
                "pn": pn,
                "gn": gn,
                "correct": pn == gn
            })
        except Exception as e:
            print(f"Error in {i}: {e}")
            print(entry['prediction'])
            print()


        # if i > 10: break
    return res


for model in ["1b", "8b"]:
    res = evaluate(f"logs/eval-{model}.log", "all")
    score = sum([1 for r in res if r["correct"]]) / len(res)
    print("Length of res:", len(res))
    print(f"Score of {model}: {score*100:.2f}%")

Error in 1335: 'NoneType' object has no attribute 'end'
' \n#### 80 +'

Error in 1357: 'NoneType' object has no attribute 'end'
' \nAnswer given: He runs 60*3=<<60*3=180>>180 miles a week\nHe runs 180*3=<<180*3=540>>540 miles\nHe runs 540/2=<<540/2=270>>270 miles a week\nHe runs 270/3=<<270/3=90>>90 miles a day\nHe runs 90*3=<<90*3=270>>270 miles a week\nHe runs 270/2=<<270/2=135>>135 miles a day\nHe runs 135*3=<<135*3=405>>405 miles a week\nHe runs 405/2=<<405/2=159>>159 miles a day\nHe runs 159*3=<<159*3=477>>477 miles a week\nHe runs 477/2=<<477/2=237>>237 miles a day\nHe runs 237*3=<<237*3=729>>729 miles a week\nHe runs 729/2=<<729/2=369>>369 miles a day\nHe runs 369*3=<<369*3=1089>>1089 miles a week\nHe runs 1089/2=<<1089/2=549>>549 miles a day\nHe runs 549*3=<<549*3=1629>>1629 miles a week\nHe runs 1629/2=<<1629/2=814>>814 miles a day\nHe runs 814*3=<<814*3=2526>>2526 miles a week\nHe runs 2526/2=<<2526/2=1261>>1261 miles a day\nHe runs 1261*3=<<1261*3=3624>>3624 miles a week\nHe

In [93]:
verifier_set = {
    "prompt": [],
    "chosen": [],
    "rejected": [],
}

instruction = "Please calculate the solution step-by-step and conclude the answer with \n#### followed by the result.\n"
for model in ["1b", "8b"]:
    res = evaluate(f"logs/eval-{model}.log", "all")
    for r in res:
        if not r['correct']:
            verifier_set['prompt'].append(f"{instruction}Question: {r['question']}")
            verifier_set['chosen'].append(r['answer'])
            verifier_set['rejected'].append(r['pred'])

Error in 1335: 'NoneType' object has no attribute 'end'
' \n#### 80 +'

Error in 1357: 'NoneType' object has no attribute 'end'
' \nAnswer given: He runs 60*3=<<60*3=180>>180 miles a week\nHe runs 180*3=<<180*3=540>>540 miles\nHe runs 540/2=<<540/2=270>>270 miles a week\nHe runs 270/3=<<270/3=90>>90 miles a day\nHe runs 90*3=<<90*3=270>>270 miles a week\nHe runs 270/2=<<270/2=135>>135 miles a day\nHe runs 135*3=<<135*3=405>>405 miles a week\nHe runs 405/2=<<405/2=159>>159 miles a day\nHe runs 159*3=<<159*3=477>>477 miles a week\nHe runs 477/2=<<477/2=237>>237 miles a day\nHe runs 237*3=<<237*3=729>>729 miles a week\nHe runs 729/2=<<729/2=369>>369 miles a day\nHe runs 369*3=<<369*3=1089>>1089 miles a week\nHe runs 1089/2=<<1089/2=549>>549 miles a day\nHe runs 549*3=<<549*3=1629>>1629 miles a week\nHe runs 1629/2=<<1629/2=814>>814 miles a day\nHe runs 814*3=<<814*3=2526>>2526 miles a week\nHe runs 2526/2=<<2526/2=1261>>1261 miles a day\nHe runs 1261*3=<<1261*3=3624>>3624 miles a week\nHe

In [75]:
from datasets import Dataset, load_dataset, load_from_disk

In [94]:
verifier_dset = Dataset.from_dict(verifier_set).train_test_split(test_size=0.1)
verifier_dset.save_to_disk("data/verifier_dataset")
len(verifier_set['prompt'])

Saving the dataset (0/1 shards):   0%|          | 0/2417 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/269 [00:00<?, ? examples/s]

2686

In [95]:
verifier_dset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 2417
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 269
    })
})

In [96]:
new = load_from_disk("data/verifier_dataset/train")

In [97]:
new[0]

{'prompt': "Please calculate the solution step-by-step and conclude the answer with \n#### followed by the result.\nQuestion: 'Julia was preparing for a dinner party at her house, where she intended to serve stew.  She noticed that she was out of plastic spoons, so she bought a new package of spoons.  Later, her husband also bought a package of 5 new spoons and gave them to Julia.  While Julia was making the stew, she used three of the spoons to sample her stew.  Later, when she went to set the table, she had a total of 12 spoons.  How many spoons were in the package that Julia bought?'",
 'chosen': '"The total number of spoons from Julia and her husband was 12+3=<<12+3=15>>15 spoons.\\nSince the husband bought a package of five spoons, then Julia\'s package contained 15-5=<<15-5=10>>10 spoons.\\n#### 10"',
 'rejected': "Julia bought 12 spoons + 3 spoons = <<12+3=15>>15 spoons\\nShe had 15 spoons + 5 spoons = <<15+5=20>>20 spoons\\nShe had 20 spoons - 3 spoons = <<20-3=17>>17 spoons\\n